<a href="https://colab.research.google.com/github/PineBiotech/bioinformatics/blob/master/Social_Bot_Twitter_PubMed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First install tweepy (all installations should only be needed once per session)

In [ ]:
 pip install tweepy

This is where app keys are set through our twitter developer profile

In [ ]:
consumer_key = "___" #api key
consumer_secret = "___" #api key secret
access_token = "___"
access_token_secret = "___"


authorization step (once per session only)

In [ ]:
import tweepy

# Authenticate to Twitter
auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)

api = tweepy.API(auth)


check authorization

In [ ]:
try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

Authentication OK


In [ ]:
# Create API object
api = tweepy.API(auth, wait_on_rate_limit=True,
    wait_on_rate_limit_notify=True)

First script is to find users based on a search term

In [ ]:
#Retweet and follow users based on search term

for tweet in api.search(q="microbiome analysis", lang="en", rpp=1):

  followstat = "now following "

  mytweet = f"RT @{tweet.user.screen_name} {tweet.text} #bioinformatics #omicslogic"
  user = ["@",tweet.user.screen_name, tweet.user.location]
  print(mytweet)
  #print(tweet.user.screen_name)
  user = api.get_user(tweet.user.screen_name)
  print(user.id)

  # Tweet something
  status = api.update_status(mytweet)

  # Like the tweet you just made
  api.create_favorite(tweet.id)

  api.create_friendship(user.id)


Find and like tweets based on a term

In [ ]:
#like a tweet based on a search term

search_string="bioinformatics education"                            #String you want to search
numberoftweets=5                                       #number of tweets you want to like
for tweet in tweepy.Cursor(api.search,search_string).items(numberoftweets):
    try:
        tweet.favorite()
        print("I liked the tweet %s"%tweet.text)  #print statement for us to know that bot has liked the tweets successfully
    except tweepy.TweepError as e:
        print(e.reason)            #exception of tweep error and also prints the error reason
    except StopIteration:
        break                        #exception for iteration error and breaks the iteration



---

This second part is to access publications on NCBI (PubMed) and prepare a short summary of a publication abstract

In [ ]:
pip install biopython

retrieve article and get link, abstract. More options: https://biopython.org/docs/1.75/api/Bio.Medline.html

In [ ]:
# Biopython to retrieve the latest article from pubmed on query term

import pandas as pd
from Bio import Medline,Entrez

#Bio.Medline.read(handle)
Entrez.email = '_____'

query = 'bioinformatics AND pipeline'
   
handle = Entrez.esearch(db="pubmed",term=query, sort="date")
record=Entrez.read(handle)
idlist=record["IdList"]

# Search, using history results if cached before
#records = Entrez.read(handle)
handle = Entrez.efetch(db="pubmed", id=idlist, rettype="medline",retmode="text")
records = Medline.parse(handle)

i = 0

for record in records:
  abstract = []
  abstract.append(record.get("AB","?"))
  author = record.get("AID","?")
  print(abstract)
  arlink1 = []
  arlink1.append("https://pubmed.ncbi.nlm.nih.gov/%s"%idlist[i])
  print(arlink1)
  #print(idlist[i])
  i = i+1

#get all links 
len(arlink1)

for i in range(len(idlist)):
  arlink1.append("https://pubmed.ncbi.nlm.nih.gov/%s"%idlist[i])

#print(arlink1)

#print only the latest link
this_id = idlist[i]

#print the last article link
arlink = "https://pubmed.ncbi.nlm.nih.gov/%s"%this_id
print(arlink)


In [ ]:
print(i)

#print only the latest link
this_id = idlist[i]

#print the last article link
arlink = "https://pubmed.ncbi.nlm.nih.gov/%s"%this_id
print(arlink1)

To summarize, we use the Google T5: Text-To-Text Transfer Transformer (https://github.com/google-research/text-to-text-transfer-transformer)

In [ ]:
pip install transformers

First load in the article abstract

In [ ]:
print(abstract)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained('t5-base')
model = AutoModelWithLMHead.from_pretrained('t5-base', return_dict=True)

inputs = tokenizer.encode("summarize: " + abstract[0],
                          return_tensors='pt',
                          max_length=1000,
                          truncation=True)


/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:882: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Now generate summary given a limited number of characters

In [ ]:
summary_ids = model.generate(inputs, max_length=30, min_length=20, length_penalty=5., num_beams=2)

summary = tokenizer.decode(summary_ids[0])

print(summary)

<pad> long-read RNA sequencing provides full-length transcripts which can be used to predict full-length protein isoforms. 


Clean up the text, add link and tweet

In [ ]:
summary = summary.replace('<pad>', '')
cleansum = summary.replace('</s>', '')

mytweet1 = "#sequencing:%s read more: %s" % (summary, arlink)
print(mytweet1)
  
# Tweet something
status = api.update_status(mytweet1)

# Like the tweet you just made
#api.create_favorite(tweet.id)

#sequencing: long-read RNA sequencing provides full-length transcripts which can be used to predict full-length protein isoforms.  read more: https://pubmed.ncbi.nlm.nih.gov/35241129
